# Задание 5. Предобработка данных и PCA
## Ершов А.Г. КМБО-03-23, 2 семестр
### Вариант 6
Набор данных: [Denver Crime Data](https://www.kaggle.com/datasets/paultimothymooney/denver-crime-data)

## 1. Сколько в наборе данных объектов и признаков? Дать описание каждому признаку, если оно есть.

Если отбросить всяческие ID, в наборе присутствует 16 признаков со следующими описаниями (нумерация идет по столбцам, столбцы, содержащие всяческие описания и ID - отброшены):

3. `offense_code` - код преступления
4. `offense_code_extension` - Расширение кода преступления (стоит "склеить" с `offence_code` или отбросить)
7. `first_occurrence_date` - Когда происшествие произошло в первый раз
8. `last_occurrence_date` - Когда происшествие произошло в последний раз относительно описываемого преступления
9. `reported_date` - Когда о происшествии было доложено в полицию
10. `incident_address` - Адрес улицы, на которой произошло происшествие
11. `geo_x` - Гео-код по оси X
12. `geo_y` - Гео-код по оси Y
13. `geo_lat` - Широта, по которой произошло происшествие
14. `geo_lon` - Долгота, по которой произошло происшествие
15. `district_id` - округ
16. `precinct_id` - участок
17. `neighborhood_id` - район
18. `is_crime` - 1, если преступление; 0, если нет
19. `is_traffic` - 1, если дорожно транспортное происшествие 0, если нет
20. `victim_count` - Число пострадавших в происшествии

## 2. Сколько категориальных признаков, какие?

Из перечисленных в пункте 1 признаков категориальными являются 

3. `offense_code` - код преступления
4. `offense_code_extension` - Расширение ID для преступления
10. `incident_address` - Адрес, по которому произошло происшествие
15. `district_id` - округ
16. `precinct_id` - участок
17. `neighborhood_id` - район

## 3. Столбец с максимальным количеством уникальных значений категориального признака?

In [2]:
import pandas as pd

data = pd.read_csv('./crime.csv', encoding='unicode_escape')
features = ['offense_code', 'offense_code_extension', 'first_occurrence_date',
            'last_occurrence_date', 'reported_date', 'incident_address',
            'geo_x', 'geo_y', 'geo_lon', 'geo_lat', 'district_id',
            'precinct_id', 'neighborhood_id', 'is_crime', 'is_traffic',
            'victim_count']
data = data.loc[:, data.columns.isin(features)]
criterial_features = ['offense_code', 'offense_code_extension',
                     'incident_address', 'district_id', 'precinct_id',
                     'neighborhood_id']
criterial_data = data.loc[:, data.columns.isin(criterial_features)]
print(f'Number of rows: {len(data)}')
criterial_data.nunique().sort_values(ascending=False)

Number of rows: 386865


incident_address          90518
offense_code                141
neighborhood_id              78
precinct_id                  41
district_id                   8
offense_code_extension        7
dtype: int64

Максимальное количество значений (90518) достигается у критериального признака `incident_addresses` (адрес, по которому произошло преступление), что не удивительно. Проводить классификацию по этому признаку, учитывая, что суммарное количество строк в датасете равно 386865 - бессмысленно.

## 4. Есть ли бинарные признаки?

Из перечисленных в пункте 1 признаков бинарными являются 

18. `is_crime` - 1, если преступление; 0, если нет
19. `is_traffic` - 1, если дорожно транспортное происшествие 0, если нет

## 5. Какие числовые признаки представлены?

Из перечисленных в пункте 1 признаков числовыми являются 

7. `first_occurence_date` - Когда происшествие произошло в первый раз
8. `last_occurrence_date` - Когда происшествие произошло в последний раз относительно описываемого преступления
9. `reported_date` - Когда о происшествии было доложено в полицию
11. `geo_x` - Гео-код по оси X
12. `geo_y` - Гео-код по оси Y
13. `geo_lat` - Широта, по которой произошло происшествие
14. `geo_lon` - Долгота, по которой произошло происшествие
20. `victim_count` - Число пострадавших в происшествии

## 6. Есть ли пропуски?

Да, в датасете присутствуют пропуски.

Количество пропусков в каждом из столбцов приведено ниже.

## 7. Сколько объектов с пропусками?

In [3]:
print(f'Total number of objects: {len(data)}')
print(f'Number of objects with NA values: {len(data) - len(data.dropna())}')
print(f'Number of \'clean\' objects: {len(data.dropna())}')

Total number of objects: 386865
Number of objects with NA values: 180345
Number of 'clean' objects: 206520


Количество объектов с пропусками оказалось равным 180345 (46.6% процентов от всех записей).

## 8. Столбец с максимальным количеством пропусков?

Столбец с максимальным количеством пропусков - `last_occurence_date` (Когда происшествие произошло в последний раз относительно описываемого преступления).
Также много пропусков в признаках, относящихся к местоположению происшествия.

In [4]:
print('Number of NA values per column:')
for col in sorted(data.columns, 
                  key=lambda x: data[x].isna().sum(), 
                  reverse=True):
    print(f'{col}: {data[col].isna().sum()}')

Number of NA values per column:
last_occurrence_date: 175556
geo_lon: 15769
geo_lat: 15769
incident_address: 15503
geo_x: 15503
geo_y: 15503
neighborhood_id: 689
district_id: 57
offense_code: 0
offense_code_extension: 0
first_occurrence_date: 0
reported_date: 0
precinct_id: 0
is_crime: 0
is_traffic: 0
victim_count: 0


## 9. Есть ли на ваш взгляд выбросы, аномальные значения?

Чтобы проверить данные на выбросы, нормализуем их. Также отделим время происшествия от даты его происхождения в отдельный признак.

In [5]:
from scipy import stats

numerical_features = ['first_occurrence_date', 'last_occurrence_date',
                      'reported_date', 'geo_x', 'geo_y', 'geo_lat', 'geo_lon',
                      'victim_count', 'reported_time', 'reported_time_utc']
numerical_data = pd.DataFrame(index=data.index, columns=numerical_features)
numerical_data.loc[:, numerical_features] = data.loc[:,
    data.columns.isin(numerical_features)]

time_features = ['first_occurrence_date',
                 'last_occurrence_date', 'reported_date']

for col in time_features:
    numerical_data.loc[:, col] = numerical_data[col].apply(
        lambda timestr: pd.to_datetime(timestr, format='%m/%d/%Y %I:%M:%S %p'))

In [6]:
numerical_data['reported_time'] = \
    numerical_data['reported_date'].apply(lambda x: pd.to_datetime(
        x.strftime('%H:%M:%S')))

In [7]:
numerical_data.loc[:, 'reported_time_utc'] = \
    numerical_data['reported_time'].dt.hour * 60 * 60 + \
    numerical_data['reported_time'].dt.minute * 60 + \
    numerical_data['reported_time'].dt.second
numerical_data = numerical_data.astype('float64', errors='ignore')

In [13]:
# Отбросим значения даты и времени в формате Timestamp для нормализации
normalizible_features = ['geo_x', 'geo_y', 'geo_lat',
                         'geo_lon', 'victim_count', 'reported_time_utc']
normalizible_data = numerical_data.loc[:, numerical_data.columns.isin(
    normalizible_features)]

for feature in normalizible_features:
    Q1 = normalizible_data[feature].quantile(0.25)
    Q3 = normalizible_data[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    normalizible_data_no_outliers = normalizible_data[
        (normalizible_data[feature] >=
         lower_bound) & (normalizible_data[feature] <= upper_bound)]
normalizible_data_no_outliers = normalizible_data_no_outliers.dropna()

print(f"Number of outliers: {len(normalizible_data) -
                             len(normalizible_data_no_outliers)}")
print(f'Total number of objects: {len(normalizible_data)}')
normalizible_data = normalizible_data_no_outliers

Number of outliers: 15769
Total number of objects: 386865


Согласно алгоритму IQR, примененному на данных, приблизительно каждый 25-ый объект - выброс. При построении моделей и классификаторов исключим их из датасета.

## 10. Столбец с максимальным средним значением после нормировки признаков через стандартное отклонение?

In [14]:
from sklearn import preprocessing

std_scaler = preprocessing.StandardScaler()
normalized_numerical_data = pd.DataFrame(
    std_scaler.fit_transform(normalizible_data.dropna()),
    columns=normalizible_data.dropna().columns,
    index=normalizible_data.dropna().index)
print("Mean values:")
for col in normalizible_features:
    print(f'{col}: {normalized_numerical_data[col].mean()}')
normalized_numerical_data.head()

Mean values:
geo_x: 5.650704189145687e-16
geo_y: -9.493152402339494e-16
geo_lat: 5.0441993575674176e-15
geo_lon: 1.0637079181535481e-14
victim_count: -3.342133424421861e-17
reported_time_utc: 8.612383926124051e-17


,geo_x,geo_y,geo_lat,geo_lon,victim_count,reported_time_utc
0,-0.043048,-0.014126,0.008177,-0.051917,-0.085388,-1.753158
1,-0.039048,0.032791,0.038181,-0.046756,-0.085388,-2.157854
2,-0.062713,0.171868,0.127624,-0.076581,-0.085388,-1.537894
3,-0.062241,-0.085401,-0.037226,-0.076405,-0.085388,-0.599344
4,0.068906,0.175905,0.128098,0.090668,-0.085388,-0.593604


Таким столбцом оказался столбец `geo_lon`, однако все полученные средние значения очень близки к нулю.

## 11. Столбец с целевым признаком?

При выполнении задачи 6 целевыми признаками для соответствующих пунктов можно выделить:

1. `district_id` - при выделении районов с наибольшим количеством преступлений
2. `offence_code` - при выделении наиболее популярных преступлений в каждом районе и при построении предсказания типа преступления по времени и месту

## 12. Сколько объектов попадает в тренировочную выборку при использовании train_test_split с параметрами test_size = 0.3, random_state = 42?

Возьмем `offence_code` как целевой признак, и разделим датасет на тренировочную и целевую выборки с указанными параметрами.

In [ ]:
from sklearn.model_selection import train_test_split

binary_features = ['is_crime', 'is_traffic']
features = criterial_features + numerical_features + binary_features
x_features = features.copy()
x_features.remove('offense_code')
x_features.remove('offense_code_extension')
target = data['offense_code']
x = data.loc[:, data.columns.isin(x_features)]

x_tr, x_val, target_tr, target_val = train_test_split(
    x, target, test_size=0.3, random_state=42)
print(f'Training split size: {len(x_tr)}')
print(f'Validation split size: {len(x_val)}')

В тренировочную выборку попало 270805 объектов, в тестовую - 116060.

## 13. Между какими признаками наблюдается линейная зависимость (корреляция)?

In [ ]:
for col in normalizible_features:
    print(normalized_numerical_data.loc[:].corr()[col][:], "\n")

Наблюдается сильная лиейная зависимость между признаками:

1. `first_occurrence_date` / `last_occurrence_date` и `reported_date`
2. `geo_lat` и `geo_y`
3. `geo_lon` и `geo_x`
4. `reported_time` и `reported_time_utc`

При дальнейшей обработке исключим некоторые из них.

## 14. Сколько признаков достаточно для объяснения 90% дисперсии после применения метода PCA?

Чтобы применить метод главных компонент, исключим мультиколлинеарные признаки, закодируем `district_id` через one-hot-encoding и объединим и закодируем `offense_code` и `offense_code_extension` через label-encoding.

In [ ]:
clean_features = ['offence_code+extension', 'reported_time_utc',
                  'geo_x', 'geo_y', 'district_id', 'is_crime',
                  'is_traffic', 'victim_count']
clean_data = pd.DataFrame(columns=clean_features, index=data.index)

clean_data['offence_code+extension'] = data['offense_code'] * \
    10 + data['offense_code_extension']
clean_data['reported_time_utc'] = numerical_data['reported_time_utc']
clean_data['geo_x'] = numerical_data['geo_x']
clean_data['geo_y'] = numerical_data['geo_y']
clean_data['district_id'] = data['district_id']
clean_data['is_crime'] = data['is_crime']
clean_data['is_traffic'] = data['is_traffic']
clean_data['victim_count'] = data['victim_count']
clean_data = clean_data.dropna()
# print(f'Number of objects in cleaned dataset: {len(clean_data.index)}')

normalizible_clean_features = ['reported_time_utc', 'geo_x', 'geo_y',
                               'victim_count']
normalizible_clean_data = clean_data.loc[:, clean_data.columns.isin(
    normalizible_clean_features)]
std_scaler = preprocessing.StandardScaler()
normalized_clean_data = pd.DataFrame(
    std_scaler.fit_transform(normalizible_clean_data),
    columns=normalizible_clean_data.columns,
    index=normalizible_clean_data.index)

for col in normalizible_clean_features:
    clean_data[col] = normalized_clean_data.loc[:, col]
clean_data = clean_data.apply(
    lambda x: pd.to_numeric(x, errors='coerce')).dropna()

clean_data.to_csv('./clean_crime.csv')

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca_data = clean_data.drop(columns=['offence_code+extension'])

pca = PCA()
x_pca = pca.fit(pca_data.values)
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)
n_components = np.argmax(cumulative_variance >= 0.9) + 1

plt.barh(pca.get_feature_names_out(), explained_variance_ratio)
plt.xlabel('Feature importances')
plt.show()

plt.bar(range(1, len(cumulative_variance) + 1), cumulative_variance)
# plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance)
plt.xlabel('Cummulative variance by number of features')
plt.show()

print(f'{n_components} features are sufficient to explain 90% of the variance.')

Как видно, 4 признака достаточно для объяснения 90% дисперсии после применения метода PCA.

## 15. Какой признак вносит наибольший вклад в первую компоненту?

In [ ]:
loadings = pca.components_[0]
feature_importances = pd.Series(
    loadings, index=pca_data.columns)
print('Feature contributions to the first component (pca0):')
print(feature_importances.abs().sort_values(ascending=False))

Признак `district_id` вносит наибольший вклад в первую компоненту.

## 16. Построить двухмерное представление данных с помощью алгоритма t-SNE. На сколько кластеров визуально, на ваш взгляд, разделяется выборка? Объяснить смысл кластеров.

In [ ]:
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
import seaborn as sns

shuffled_data = shuffle(clean_data, random_state=0).head(20000)

x = shuffled_data.drop(columns=['offence_code+extension',])
y = shuffled_data.loc[:, clean_data.columns.isin(['offence_code+extension'])]

z = TSNE(n_components=2, learning_rate='auto', random_state=0,
         init='pca', perplexity=50).fit_transform(x)

x_features = clean_features.copy()
x_features.remove('offence_code+extension')

df = pd.DataFrame()
df['y'] = y
df['Component 1'] = z[:, 0]
df['Component 2'] = z[:, 1]


sns.scatterplot(x='Component 1', y='Component 2',
                hue=shuffled_data['district_id'],
                data=df).set(title='t-SNE Visualization (district_id)')
plt.show()
sns.scatterplot(x='Component 1', y='Component 2',
                hue=shuffled_data['victim_count'],
                data=df).set(title='t-SNE Visualization (victim_count)')
plt.show()
sns.scatterplot(x='Component 1', y='Component 2',
                hue=shuffled_data['offence_code+extension'],
                data=df).set(
                    title='t-SNE Visualization (offence_code+extension)')
plt.show()

На полученных графиках четко видно, что данные разделяются на 8 кластеров, 7 из которых представляют собой районы (`district_id`). Восьмой кластер (3-ий график сверху) - преступления c большим количесвом жертв, (их можно посмотреть в выводе кода ниже) которые имеют похожий код `131**` - различные виды массовой стрельбы.

In [ ]:
clean_data.sort_values(by=['victim_count'],
                       ascending=False).loc[:,
                        clean_data.columns.isin(['offence_code+extension', 'victim_count'])].head(10)